In [1]:
!pip install -U openmim
!mim install mmengine

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.4.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.3/452.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 24.0 MB/s eta 0:00:00


In [ ]:
!pip install mmcv==2.1.0
!mim install "mmdet==3.2.0"
!mim install "mmpose>=1.1.0"

  Using cached mmcv-2.1.0.tar.gz (471 kB)
  Preparing metadata (setup.py) ... done


In [ ]:
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
!pip install -r requirements.txt
!pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in editable mode,
# thus any local modifications made to the code will take effect without reinstallation.

In [ ]:
!wget https://download.openmmlab.com/mmpose/v1/projects/rtmo/rtmo-m_16xb16-600e_coco-640x640-6f4e0306_20231211.pth

In [ ]:
%cd ..
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack

In [ ]:
!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox onemetric loguru lap thop==0.1.1.post2209072238
from IPython import display
display.clear_output()

In [40]:
import sys
sys.path.append('/content/ByteTrack')
sys.path.append('/content/mmpose')

In [1]:
%cd mmpose

/teamspace/studios/this_studio/mmpose


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import cv2
import torch
from typing import Optional, List, Dict
from dataclasses import dataclass
from yolox.tracker.byte_tracker import STrack, BYTETracker
from onemetric.cv.utils.iou import box_iou_batch
from mmpose.apis import MMPoseInferencer
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.float = np.float64
import os
import time
import glob
from tqdm.notebook import tqdm

In [3]:
@dataclass
class Detection:
    x_min: float
    y_min: float
    x_max: float
    y_max: float
    keypoints: np.ndarray
    keypoint_scores: np.ndarray
    confidence: float
    tracker_id: Optional[int] = None

    @classmethod
    def from_numpy(cls, pred: np.ndarray, keypoints_list: List[np.ndarray], keypoint_scores_list: List[np.ndarray]):
        result = []
        for (x_min, y_min, x_max, y_max, confidence), keypoints, keypoint_scores in zip(pred, keypoints_list, keypoint_scores_list):
            result.append(Detection(
                x_min = float(x_min),
                y_min = float(y_min),
                x_max = float(x_max),
                y_max = float(y_max),
                keypoints = keypoints,
                keypoint_scores = keypoint_scores,
                confidence=float(confidence)
            ))
        return result

@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

# converts List into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: List[Detection], with_confidence: bool = True) -> np.ndarray:
    return np.array([
        [
            detection.x_min,
            detection.y_min,
            detection.x_max,
            detection.y_max,
            detection.confidence
        ] if with_confidence else [
            detection.x_min,
            detection.y_min,
            detection.x_max,
            detection.y_max,
        ]
        for detection
        in detections
    ], dtype=float)
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=np.float64)

def match_detections_with_tracks(
    detections: List[Detection],
    tracks: List[STrack]
) -> List:
    if not np.any(detections) or len(tracks) == 0:
        return np.empty((0,))
    detection_boxes = detections2boxes(detections=detections, with_confidence=False)
    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detection_boxes)
    track2detection = np.argmax(iou, axis=1)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            detections[detection_index].tracker_id = tracks[tracker_index].track_id
    return detections

In [4]:

def generate_joint_heatmaps(keypoints, heatmap_size, sigma=0.6):
    """
    Generate a heatmap for each keypoint and stack them into a 3D volume (KxHxW).

    Args:
    - keypoints: numpy array of shape (K, 3), where K is the number of keypoints,
                 and each keypoint is represented as (x, y, c) for coordinates and confidence score.
    - heatmap_size: tuple (H, W), the size of each heatmap (height, width).
    - sigma: standard deviation (spread) of the Gaussian.

    Returns:
    - heatmaps: numpy array of shape (K, H, W), the stacked 3D heatmaps.
    """
    K = keypoints.shape[0]
    H, W = heatmap_size

    # Initialize the 3D heatmap volume: shape (K, H, W)
    heatmaps = np.zeros((K, H, W), dtype=np.float32)

    # Create a meshgrid for the heatmap
    x = np.arange(0, W, 1)
    y = np.arange(0, H, 1)
    xx, yy = np.meshgrid(x, y)

    # Generate a heatmap for each keypoint
    for k in range(K):
        xk, yk, ck = keypoints[k]  # Extract (x, y, confidence) for keypoint k
        # Apply the Gaussian formula from the equation
        gaussian_map = np.exp(-((xx - xk) ** 2 + (yy - yk) ** 2) / (2 * sigma ** 2)) * ck
        heatmaps[k] = gaussian_map  # Store the heatmap for keypoint k

    return heatmaps

def distance_to_segment(px, py, ax, ay, bx, by):
    """
    Compute the distance from point (px, py) to the line segment between (ax, ay) and (bx, by).
    Vectorized operation for entire heatmap at once.
    """
    # Vectorize the line segment length
    segment_length_sq = (bx - ax) ** 2 + (by - ay) ** 2

    # Prevent division by zero in case a and b are the same points
    segment_length_sq = np.maximum(segment_length_sq, 1e-8)

    # Projection of the point onto the line segment
    t = np.clip(((px - ax) * (bx - ax) + (py - ay) * (by - ay)) / segment_length_sq, 0, 1)

    # Projection coordinates on the line segment
    proj_x = ax + t * (bx - ax)
    proj_y = ay + t * (by - ay)

    # Distance from point (px, py) to the projection
    return np.sqrt((px - proj_x) ** 2 + (py - proj_y) ** 2)

def generate_limb_heatmaps(limb_pairs, heatmap_size, sigma=0.6):
    """
    Generate limb heatmaps for all limb pairs and stack them into a 3D volume (KxHxW).

    Args:
    - limb_pairs: numpy array of shape (K, 2, 3), where K is the number of limbs,
                  and each limb is represented by two keypoints ((x_a, y_a, c_a), (x_b, y_b, c_b)).
    - heatmap_size: tuple (H, W), the size of each heatmap (height, width).
    - sigma: standard deviation (spread) of the Gaussian.

    Returns:
    - heatmaps: numpy array of shape (K, H, W), the stacked 3D heatmaps for limbs.
    """
    K = limb_pairs.shape[0]
    H, W = heatmap_size

    # Initialize the 3D heatmap volume: shape (K, H, W)
    heatmaps = np.zeros((K, H, W), dtype=np.float32)

    # Create a meshgrid for the heatmap
    x = np.arange(0, W, 1)
    y = np.arange(0, H, 1)
    xx, yy = np.meshgrid(x, y)

    for k in range(K):
        # Extract coordinates and confidence scores for the two keypoints forming the limb
        (xa, ya, ca), (xb, yb, cb) = limb_pairs[k]

        # Compute the distance of each point in the heatmap to the limb segment
        distances = distance_to_segment(xx, yy, xa, ya, xb, yb)

        # Apply the Gaussian formula
        gaussian_map = np.exp(-distances ** 2 / (2 * sigma ** 2))

        # Multiply by the minimum confidence between the two keypoints
        confidence = min(ca, cb)
        heatmaps[k] = gaussian_map * confidence

    return heatmaps

def generate_limb_pairs(keypoints, connections):
    """
    Generate limb pairs for each connection of keypoints.

    Args:
    - keypoints: numpy array of shape (K, 3), where K is the number of keypoints,
                 and each keypoint is represented as (x, y, c).
    - connections: list of tuples, where each tuple contains two indices representing a connection
                   between two keypoints (e.g., (a, b)).

    Returns:
    - limb_pairs: numpy array of shape (L, 2, 3), where L is the number of limbs,
                  and each limb contains two keypoints represented as (x, y, c).
    """
    limb_pairs = []

    for (a, b) in connections:
        if a < keypoints.shape[0] and b < keypoints.shape[0]:
            # Extract the keypoints corresponding to the connection
            point_a = keypoints[a]
            point_b = keypoints[b]
            limb_pairs.append([point_a, point_b])

    return np.array(limb_pairs)

byte_tracker = BYTETracker(BYTETrackerArgs())
connections = [
    (0, 1), (0, 2), (2, 4), (1, 3), (0,5), (0,6), (5,7), (7,9), (6,8), (8,10), (5,11),
    (6,12), (11,12), (12,14), (11,13), (13,15), (14,16)
]

def extract_keypoints_and_create_heatmaps(inferencer, video_path, heatmap_size):
    """Extract keypoints from video and generate heatmaps."""
    result_generator = inferencer(video_path, show=False)
    results = [result for result in result_generator]
    dict_person = {}
    vcap = cv2.VideoCapture(video_path)
    width = vcap.get(3)
    height = vcap.get(4)
    frame_shape = (width,height)
    for frame_id, result in enumerate(results):
      bboxs = np.array([])
      keypoints_list = []
      keypoint_scores_list = []
      # frame_draw = Image.fromarray(frame)
      if len(result['predictions'][0]) != 0:
            for i, res in enumerate(result['predictions'][0]):
              # bounding box are xmin, ymin, xmax, ymax
              if res['bbox_score'] > 0.45 and np.mean(res['keypoint_scores']) > 0.45:
                  box = np.array(res['bbox'][0],dtype = np.float32)
                  # crop_frame = frame[int(box[1]) : int(box[3]), int(box[0]) : int(box[2])]
                  if len(bboxs) == 0:
                      box = np.append(box,res['bbox_score'])
                      bboxs = box
                  else:
                      box = np.append(box,res['bbox_score'])
                      bboxs = np.vstack((bboxs,box))
                  keypoints = res['keypoints']
                  adjusted_keypoints = []
                  for point in keypoints:
                      adjusted_keypoints.append([point[0], point[1]])
                  keypoints_list.append(np.array(adjusted_keypoints))
                  keypoint_scores_list.append(res['keypoint_scores'])
            if len(bboxs) > 0:
              if bboxs.ndim == 1:
                  bboxs = np.expand_dims(bboxs,0)
              detections = Detection.from_numpy(pred=bboxs.copy(), keypoints_list = keypoints_list.copy(), keypoint_scores_list=keypoint_scores_list)
              tracks = byte_tracker.update(
                      output_results=bboxs,
                      img_info=frame_shape,
                      img_size=frame_shape
              )
              tracked_detections = match_detections_with_tracks(detections=detections.copy(), tracks=tracks.copy())
              detection_boxes = detections2boxes(detections=tracked_detections, with_confidence=False)
              if len(tracked_detections) > 0:
                  for i, tracked_detection in enumerate(tracked_detections):
                      if tracked_detection.tracker_id:
                          # Generate joint heatmaps and limb heatmaps
                          keypoints_w_conf = tracked_detection.keypoints.copy()
                          keypoints_w_conf = np.append(keypoints_w_conf, np.array(tracked_detection.keypoint_scores).reshape(-1,1),1)
                          joint_heatmap = generate_joint_heatmaps(keypoints_w_conf, heatmap_size)

                          limb_pairs = generate_limb_pairs(keypoints_w_conf, connections)
                          limb_heatmap = generate_limb_heatmaps(limb_pairs, heatmap_size)
                          if tracked_detection.tracker_id not in dict_person:
                              # Initialize the joint and limb heatmaps as np.array with shape (1, 17, 56, 56)
                              dict_person[tracked_detection.tracker_id] = {
                                  'joint_heatmaps': joint_heatmap[np.newaxis, ...],  # Shape (1, 17, 56, 56)
                                  'limb_heatmaps': limb_heatmap[np.newaxis, ...],    # Shape (1, 17, 56, 56)
                                  'bboxs': [[tracked_detection.x_min, tracked_detection.y_min, tracked_detection.x_max, tracked_detection.y_max]],  # Store bbox as list
                                  'keypoints': [tracked_detection.keypoints],         # List of keypoints for each frame
                                  'frame_ids': [frame_id]  # Store current frame ID
                              }
                          else:
                              # Concatenate the new heatmap along the time axis (axis=0)
                              dict_person[tracked_detection.tracker_id]['joint_heatmaps'] = np.concatenate(
                                  (dict_person[tracked_detection.tracker_id]['joint_heatmaps'], joint_heatmap[np.newaxis, ...]), axis=0
                              )
                              dict_person[tracked_detection.tracker_id]['limb_heatmaps'] = np.concatenate(
                                  (dict_person[tracked_detection.tracker_id]['limb_heatmaps'], limb_heatmap[np.newaxis, ...]), axis=0
                              )
                              dict_person[tracked_detection.tracker_id]['bboxs'].append([tracked_detection.x_min, tracked_detection.y_min, tracked_detection.x_max, tracked_detection.y_max])
                              dict_person[tracked_detection.tracker_id]['keypoints'].append(tracked_detection.keypoints)
                              dict_person[tracked_detection.tracker_id]['frame_ids'].append(frame_id)

    return dict_person


In [88]:

# def uniform_sample_clip(video, T=48):
#     """
#     Uniformly sample T frames from a video by dividing the video into T segments
#     and randomly selecting one frame from each segment.

#     Args:
#         video (torch.Tensor): Tensor containing video frames with shape (Total_frames, C, H, W).
#         T (int): Number of frames to sample (i.e., the number of segments to divide the video into).

#     Returns:
#         torch.Tensor: A tensor of shape (T, C, H, W) containing the sampled frames.
#     """
#     # Total number of frames in the video
#     total_frames = video.shape[0]

#     # Check if T is valid
#     if T <= 0 or T > total_frames:
#         raise ValueError(f"T must be between 1 and {total_frames}, but got T={T}.")

#     # Calculate the length of each segment (using integer division)
#     segment_length = total_frames // T

#     sampled_frames = []

#     # Sample one random frame from each segment
#     for i in range(T):
#         start = i * segment_length
#         end = (i + 1) * segment_length if i < T - 1 else total_frames  # Last segment goes to total_frames

#         # Randomly choose a frame within this segment
#         random_index = torch.randint(start, end, (1,)).item()

#         # Append the selected frame to the result list
#         sampled_frames.append(video[random_index])

#     # Stack the sampled frames into a tensor of shape (T, C, H, W)
#     sampled_clip = np.array(sampled_frames)

#     return sampled_clip

def uniform_sample_clip(video, T):
    """
    Uniformly sample T frames from a video by dividing the video into T segments
    and randomly selecting one frame from each segment.

    Args:
        video (torch.Tensor): Tensor containing video frames with shape (Total_frames, C, H, W).
        T (int): Number of frames to sample (i.e., the number of segments to divide the video into).

    Returns:
        torch.Tensor: A tensor of shape (T, C, H, W) containing the sampled frames.
    """
    # Total number of frames in the video
    total_frames = video.shape[0]

    # Calculate the length of each segment
    segment_length = total_frames / T

    sampled_frames = []

    # Sample one random frame from each segment
    for i in range(T):
        start = int(i * segment_length)
        end = int((i + 1) * segment_length)

        # Randomly choose a frame within this segment
        random_index = torch.randint(start, end, (1,)).item()

        # Append the selected frame to the result list
        sampled_frames.append(video[random_index])

    # Stack the sampled frames into a tensor of shape (T, C, H, W)
    sampled_clip = torch.stack(sampled_frames)

    return sampled_clip

def sample_person_heatmaps(dict_person, min_T = 32, T=48, scale=6, method='duplicate'):
    """
    Uniformly sample 48 heatmaps from every 288 frames for each person in dict_person.
    
    Args:
        dict_person (dict): Dictionary containing heatmaps for each tracked person.
        T (int): Number of frames to sample.
        scale (int): The scale factor for frame sampling.
        method (str): Method to use when frames are less than T: 'duplicate' or 'interpolate'.
        
    Returns:
        dict: A new dictionary with sampled heatmaps of shape (T, 17, 56, 56).
    """
    sampled_dict_person = {}
    total_frames_threshold = int(T * scale)

    for person_id, data in dict_person.items():
        # Get the joint and limb heatmaps
        joint_heatmaps = data['joint_heatmaps']  # Shape (Total_frames, 17, 56, 56)
        limb_heatmaps = data['limb_heatmaps']    # Shape (Total_frames, 17, 56, 56)
        total_frames = joint_heatmaps.shape[0]

        # Prepare lists to store sampled heatmaps
        sampled_joint_heatmaps = []
        sampled_limb_heatmaps = []
        batch_joint_heatmaps = np.array([])

        # Process frames in batches of 288
        if scale == 0 or total_frames_threshold >= total_frames:
            if total_frames >= T:
                sampled_joint = uniform_sample_clip(torch.tensor(joint_heatmaps), T)
                sampled_limb = uniform_sample_clip(torch.tensor(limb_heatmaps), T)
            elif total_frames > min_T:
                if method == 'duplicate':
                    # Duplicate frames to make up the difference
                    sampled_joint = np.resize(joint_heatmaps, (T, *joint_heatmaps.shape[1:]))
                    sampled_limb = np.resize(limb_heatmaps, (T, *limb_heatmaps.shape[1:]))
                elif method == 'interpolate':
                    # Interpolate between frames
                    sampled_joint = interpolate_heatmaps(joint_heatmaps, T)
                    sampled_limb = interpolate_heatmaps(limb_heatmaps, T)
            elif total_frames < min_T:
                    sampled_joint = np.array([])
                    sampled_limb = np.array([])
        else:
            for start in range(0, total_frames, total_frames_threshold):
                end = min(start + total_frames_threshold, total_frames)  # Handle remaining frames
                batch_joint_heatmaps = joint_heatmaps[start:end]
                batch_limb_heatmaps = limb_heatmaps[start:end]

                # If the batch has enough frames, sample uniformly
                if batch_joint_heatmaps.shape[0] >= total_frames_threshold or batch_joint_heatmaps.shape[0] >= T:
                    # Use the uniform_sample_clip function to sample T frames from this batch
                    sampled_joint = uniform_sample_clip(torch.tensor(batch_joint_heatmaps), T)
                    sampled_limb = uniform_sample_clip(torch.tensor(batch_limb_heatmaps), T)
                elif batch_joint_heatmaps.shape[0] > min_T:
                    # If not enough frames, handle based on the chosen method
                    if method == 'duplicate':
                        # Duplicate frames to make up the difference
                        sampled_joint = np.resize(batch_joint_heatmaps, (T, *batch_joint_heatmaps.shape[1:]))
                        sampled_limb = np.resize(batch_limb_heatmaps, (T, *batch_limb_heatmaps.shape[1:]))
                    elif method == 'interpolate':
                        # Interpolate between frames
                        sampled_joint = interpolate_heatmaps(batch_joint_heatmaps, T)
                        sampled_limb = interpolate_heatmaps(batch_limb_heatmaps, T)
                    else:
                        raise ValueError(f"Unknown method '{method}'. Use 'duplicate' or 'interpolate'.")
                else:
                    sampled_joint = np.array([])
                    sampled_limb = np.array([])
        if len(sampled_joint) > 0:
            # Append to sampled lists
            if len(batch_joint_heatmaps)>0:
                print(batch_joint_heatmaps.shape,'->',sampled_joint.shape)
            else:
                print(joint_heatmaps.shape,'->',sampled_joint.shape)
            sampled_joint_heatmaps.append(sampled_joint)
            sampled_limb_heatmaps.append(sampled_limb)

        # Concatenate all sampled heatmaps
        # sampled_joint_heatmaps = np.concatenate(sampled_joint_heatmaps, axis=0)
        # sampled_limb_heatmaps = np.concatenate(sampled_limb_heatmaps, axis=0)
        
        # Copy the bbox and keypoints as they don't need sampling
        bboxs = data['bboxs']
        keypoints = data['keypoints']
        frame_ids = data['frame_ids']  # Store current frame ID

        # Store the sampled heatmaps and other data in a new dictionary
        sampled_dict_person[person_id] = {
            'joint_heatmaps': sampled_joint_heatmaps,  # Already in numpy format
            'limb_heatmaps': sampled_limb_heatmaps,    # Already in numpy format
            'bboxs': bboxs,  # Keep bboxs and keypoints unchanged
            'keypoints': keypoints,
            'frame_ids': frame_ids
        }

    return sampled_dict_person

def interpolate_heatmaps(heatmaps, target_frames):
    """
    Interpolates heatmaps to match the target number of frames.
    
    Args:
        heatmaps (np.ndarray): Heatmaps of shape (frames, 17, 56, 56).
        target_frames (int): The desired number of frames.
        
    Returns:
        np.ndarray: Interpolated heatmaps of shape (target_frames, 17, 56, 56).
    """
    current_frames = heatmaps.shape[0]
    factor = target_frames / current_frames
    interpolated_heatmaps = np.zeros((target_frames, *heatmaps.shape[1:]))

    for i in range(target_frames):
        # Map new frame to old frame indices
        old_frame_idx = i / factor
        lower_idx = int(np.floor(old_frame_idx))
        upper_idx = min(int(np.ceil(old_frame_idx)), current_frames - 1)

        # Linear interpolation between two closest frames
        weight = old_frame_idx - lower_idx
        interpolated_heatmaps[i] = (1 - weight) * heatmaps[lower_idx] + weight * heatmaps[upper_idx]

    return interpolated_heatmaps


In [89]:
def run_inference(heatmaps, model):
    """Run inference on the heatmaps using the 3D-CNN model."""
    # heatmaps_tensor = torch.tensor(heatmaps).unsqueeze(0)  # Add batch dim (1, 17, T, 56, 56)
    # print(heatmaps_tensor.shape)
    heatmaps_tensor = torch.tensor(heatmaps,dtype=torch.float)
    heatmaps_tensor = heatmaps_tensor.permute(1,0,2,3).unsqueeze(0)
    # print(heatmaps_tensor.shape)
    with torch.no_grad():
        model.eval()  # Set model to evaluation mode
        output = model(heatmaps_tensor)
        output = output.squeeze(0).cpu().numpy()  # Remove batch dim and move to CPU
        print(output)
        output = np.argmax(output, axis=0).astype(int)  # Get the class with highest probability
    label = 'Normal' if output == 0 else 'Shoplifting'
    return label

def run_inference_on_sampled_dict(sampled_dict_person, model, heatmap_type='joint'):
    """
    Run inference on the sampled heatmaps for each person in sampled_dict_person and
    include bboxs and keypoints in the results.

    Args:
        sampled_dict_person (dict): Dictionary containing the sampled heatmaps for each tracked person.
        model: The 3D-CNN model for inference.
        heatmap_type (str): The type of heatmaps to use for inference. Can be 'joint', 'limb', or 'both'.

    Returns:
        dict: A dictionary with the inference results, bboxs, and keypoints for each person.
    """
    inference_results = {}

    for person_id, data in sampled_dict_person.items():
        # Initialize a list to hold predictions for each clip
        predictions = []

        # Select the appropriate heatmaps based on heatmap_type
        if heatmap_type == 'joint':
            heatmaps_list = data['joint_heatmaps']
        elif heatmap_type == 'limb':
            heatmaps_list = data['limb_heatmaps']
        elif heatmap_type == 'both':
            # Combine corresponding clips of joint and limb heatmaps
            heatmaps_list = [j + l for j, l in zip(data['joint_heatmaps'], data['limb_heatmaps'])]
        else:
            raise ValueError("Invalid heatmap_type. Must be 'joint', 'limb', or 'both'.")

        # Run inference on each heatmap clip and collect predictions
        for heatmaps in heatmaps_list:
            prediction = run_inference(heatmaps, model)  # Each heatmap is of shape (T, 17, 56, 56)
            # prediction = [prediction]*heatmaps.shape[0]
            predictions.append(prediction)

        # Store the result along with bbox and keypoints
        if len(predictions) > 0:
            inference_results[person_id] = {
                'predictions': predictions,  # List of predictions for each T-frame clip
                'bboxs': data['bboxs'],
                'keypoints': data['keypoints'],
                'frame_ids': data['frame_ids']
            }

    return inference_results

def visualize_inference_on_video(video_path, inference_results, output_path='prediction.mp4', fps=30, T = 48, scale = 6):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    frame_selected = int(T*scale)
    if scale == 0:
        frame_selected = total_frames

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Check for each person in the inference results
        for person_id, result in inference_results.items():
            # Check if there are predictions for this person
            if frame_count in result['frame_ids']:
                # Find the current clip index based on frame_count
                clip_index = frame_count // frame_selected
                bbox_id = result['frame_ids'].index(frame_count)

                # Ensure clip_index does not exceed available predictions
                if clip_index < len(result['predictions']):
                    prediction = result['predictions'][clip_index]  # Get the prediction for the current clip
                    bbox = result['bboxs'][bbox_id]  # Get the corresponding bbox for this clip
                    keypoints = result['keypoints'][bbox_id]  # Get the corresponding keypoints

                    # Determine label and color
                    color = (0, 255, 0) if prediction == 'Normal' else (0, 0, 255) 

                    # Draw bounding box
                    x_min, y_min, x_max, y_max = bbox
                    cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color, 2)
                    cv2.putText(frame, f'{prediction}',
                                (int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                    # Draw keypoints (if needed)
                    # for (x, y) in keypoints:  # Assuming keypoints are available and valid
                    #     cv2.circle(frame, (int(x), int(y)), 3, color, -1)

        # Write the processed frame to the output video
        out.write(frame)

        frame_count += 1

    cap.release()
    out.release()

    # cv2.destroyAllWindows()

In [90]:
def process_input(model=None, video_source=None, inferencer=None, heatmap_size=(56, 56), heatmap_type='joint', 
                visualize=False, output_path='output_video.mp4', fps=30, min_T=32, T=64, scale=5, method='interpolate'):
    """
    Process input for inference.
    If video_source is provided and valid, it processes the video file.
    If video_source is None or 0, it processes live camera stream.

    Args:
        model: The 3D-CNN model for inference.
        video_source: The source of the video (0 for camera, filename for video).
        inferencer: The object or function to extract keypoints and create heatmaps.
        heatmap_size: The size of the heatmaps.
        heatmap_type: Type of heatmap to process ('joint', 'limb', or 'both').
        visualize: Flag to indicate whether to visualize the output.
        output_path: The path to save the output video if visualizing.
    """
    if video_source is None or video_source == 0:
        # Real-time inference from camera
        cap = cv2.VideoCapture(0)  # Camera stream
        frames = []
        dict_person = {}

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frames.append(frame)

            if len(frames) >= 48:
                # Process frames to extract heatmaps
                dict_person = extract_keypoints_and_create_heatmaps(inferencer, video_source, heatmap_size)
                dict_person = sample_person_heatmaps(dict_person, min_T, T,scale,method)

                # Run inference
                dict_pred = run_inference_on_sampled_dict(dict_person, model, heatmap_type)

                # Print predictions for each person
                for person_id, prediction in dict_pred.items():
                    print(f"Person ID {person_id}: Prediction: {dict_pred[person_id]['predictions']}")

                # Visualization
                if visualize:
                    visualize_inference_on_video(None, dict_person, output_path,fps,T,scale)  # Adjust as necessary

                frames = []  # Reset for next batch

        cap.release()

    else:
        # Process video file
        print(f'--------Extracting keypoints and generate heatmap for {heatmap_type}--------')
        dict_person = extract_keypoints_and_create_heatmaps(inferencer, video_source, heatmap_size)
        dict_person = sample_person_heatmaps(dict_person, min_T, T,scale,method)
        print(f'--------Runing inference on heatmap--------')
        # Run inference
        dict_pred = run_inference_on_sampled_dict(dict_person, model, heatmap_type)
        print(f'--------Prediction--------')
        # Print predictions for each person
        for person_id, prediction in dict_pred.items():
            print(f"Person ID {person_id} -> Prediction: {dict_pred[person_id]['predictions']}")

        # Visualization
        if visualize:
            print(f'--------Result save on {output_path}--------')
            visualize_inference_on_video(video_source, dict_pred, output_path, fps,T,scale)
    return dict_pred

In [78]:

# inferencer = MMPoseInferencer('human')
config_file = 'configs/body_2d_keypoint/rtmo/coco/rtmo-m_16xb16-600e_coco-640x640.py'
checkpoint_file = 'rtmo-m_16xb16-600e_coco-640x640-6f4e0306_20231211.pth'

# build the inferencer with model config path and checkpoint path/URL
inferencer = MMPoseInferencer(
    pose2d=config_file,
    pose2d_weights=checkpoint_file,
    device= 'cpu'
)

Loads checkpoint by local backend from path: rtmo-m_16xb16-600e_coco-640x640-6f4e0306_20231211.pth


In [9]:
!gdown 1nY7jezbQzLf_SepALLpzafGjL0YShmjS

Downloading...
From (original): https://drive.google.com/uc?id=1nY7jezbQzLf_SepALLpzafGjL0YShmjS
From (redirected): https://drive.google.com/uc?id=1nY7jezbQzLf_SepALLpzafGjL0YShmjS&confirm=t&uuid=b072d212-709d-443d-a5ff-9ee145eed4ec
To: /teamspace/studios/this_studio/mmpose/Weights.zip
100%|████████████████████████████████████████| 124M/124M [00:04<00:00, 31.0MB/s]


In [10]:
!unzip Weights.zip

Archive:  Weights.zip
   creating: Weights/
  inflating: Weights/best_model_c3d_both.pth  
  inflating: Weights/best_model_c3d_joint.pth  
  inflating: Weights/best_model_c3d_limb.pth  
  inflating: Weights/best_model_x3d_both.pth  
  inflating: Weights/best_model_x3d_joint.pth  
  inflating: Weights/best_model_x3d_limb.pth  
  inflating: Weights/rtmo-coco-640x640.pth  


In [79]:
# Script to write the config file in Jupyter Notebook cell

config_content = """
# model = dict(
#     type='Recognizer3D',
#     backbone=dict(
#         type='X3D',
#         gamma_d=1,
#         in_channels=17,
#         base_channels=24,
#         num_stages=3,
#         se_ratio=None,
#         use_swish=False,
#         stage_blocks=(2, 5, 3),
#         spatial_strides=(2, 2, 2)),
#     cls_head=dict(
#         type='I3DHead',
#         in_channels=216,
#         num_classes=60,
#         dropout=0.5),
#     test_cfg=dict(average_clips='prob'))
model = dict(
    type='Recognizer3D',
    backbone=dict(
        type='C3D',
        in_channels=17,
        base_channels=32,
        num_stages=3,
        temporal_downsample=False),
    cls_head=dict(
        type='I3DHead',
        in_channels=256,
        num_classes=2,
        dropout=0.5),
    test_cfg=dict(average_clips='prob'))

# lr_new =  lr / (num_gpu * batch_size_per_gpu) (lr=0.4 | 8 gpus)
optimizer=dict(
    type='SGD',
    lr=0.00625,
    momentum=0.9,
    weight_decay=0.0003)

optimizer_config=dict(
    grad_clip=dict(max_norm=40, norm_type=2))

# Learning policy
lr_config=dict(
    policy='CosineAnnealing',
    by_epoch=False,
    min_lr=0)

total_epochs=24

checkpoint_config=dict(
    interval=1)
# Dataset configuration
dataset = dict(
    type='HeatmapVideoDataset',
    root_dir='./Heatmap_Dataset',
    num_frames=64,
    heatmap_type='joint',
    heatmap_size=(17, 56, 56),
    test_size = 0.2,
    batch_size = 32
)

transform = dict(
  type='DropKeypoint',
  drop_prob=0.125
)
"""

# Specify the file path and name
config_file_path = 'model_config.py'

# Write the content to the file
with open(config_file_path, 'w') as f:
    f.write(config_content)

print(f"Config file written to: {config_file_path}")


Config file written to: model_config.py


In [80]:
from model import load_config, build_model
config = load_config("model_config.py")

In [81]:
model_cfg = config.model
model = build_model(model_cfg)

In [82]:
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.device_count() > 1:
    print(f"Sử dụng {torch.cuda.device_count()} GPUs")
# model = nn.DataParallel(model)

model = model.to(device)

In [92]:
model.load_state_dict(torch.load("./Weights/best_model_c3d_joint.pth",map_location=torch.device('cpu')))


<All keys matched successfully>

In [63]:
print(model)

Recognizer3D(
  (backbone): C3D(
    (conv1a): ConvBlock(
      (block): Sequential(
        (0): Conv3d(17, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (pool1): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (conv2a): ConvBlock(
      (block): Sequential(
        (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (pool2): AvgPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0)
    (conv3a): ConvBlock(
      (block): Sequential(
        (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplac

In [91]:
dict_pred = process_input(
    model = model,
    video_source='./Shoplifting_Dataset/shoplifting/shop_lifter_0.mp4',
    inferencer=inferencer,
    heatmap_size=(56, 56),
    heatmap_type='joint', 
    visualize=True, 
    output_path='output_video.mp4',
    fps = 24.75,
    min_T = 64,
    T = 64,
    scale = 0,
    method = 'interpolate'
)

--------Extracting keypoints and generate heatmap for joint--------


(384, 17, 56, 56) -> torch.Size([64, 17, 56, 56])
--------Runing inference on heatmap--------
[-0.30702993  0.28088185]
--------Prediction--------
Person ID 5 -> Prediction: ['Shoplifting']
--------Result save on output_video.mp4--------
